Compute the KS test to determine whether the void and wall samples are drawn from the same or different parent distributions.

In [1]:
from astropy.table import Table
from astropy.io import fits

import numpy as np

import sys

import pickle

import dynesty
from dynesty import plotting as dyplot

import matplotlib
import matplotlib.pyplot as plt

from functions import Model_1_fit, Model_2_fit, Model_1_plot, Model_2_plot, Model_1_sampler, Model_2_sampler, uniform, jeffreys

from scipy.stats import ks_2samp
#from scipy import stats
#from scipy.optimize import curve_fit
#from scipy.stats import skewnorm

np.set_printoptions(threshold=sys.maxsize)

In [2]:
#matplotlib.rcParams['text.usetex']=True
matplotlib.rc('font', size=14)
#matplotlib.rc('text', usetex=True)
matplotlib.rc('font', family='DejaVu Sans')

# Data

In [3]:
data_filename = '../../../../data/NSA_v1_0_1_VAGC_vflag-V2-VF.fits'

hdu = fits.open(data_filename)
data = Table(hdu[1].data)
hdu.close()

## Just keep the main SDSS DR7 footprint

In [4]:
# When the catalog is the NSA, only keep those objects which were originally in the DR7 LSS
catalog_SDSS = data[data['IN_DR7_LSS'] == 1]

ra_boolean = np.logical_and(catalog_SDSS['RA'] > 110, catalog_SDSS['RA'] < 270)
catalog_north = catalog_SDSS[ra_boolean]

strip_boolean = np.logical_and.reduce([catalog_north['RA'] > 250, 
                                       catalog_north['RA'] < 269, 
                                       catalog_north['DEC'] > 51, 
                                       catalog_north['DEC'] < 67])
catalog_main = catalog_north[~strip_boolean]

In [5]:
################################################################################
# stellar mass
#-------------------------------------------------------------------------------
Mstar_NSA = np.log10(catalog_main['ELPETRO_MASS'])
################################################################################


################################################################################
# color
#-------------------------------------------------------------------------------
u_r_NSA = np.array(catalog_main['u_r'])

g_r_NSA = np.array(catalog_main['g_r'])
################################################################################


################################################################################
# absolute magnitude
#-------------------------------------------------------------------------------
rabsmag_NSA = np.array(catalog_main['ELPETRO_ABSMAG'][:,4])
################################################################################


################################################################################
# star formation rate and specific SPR
#-------------------------------------------------------------------------------
SFR  = np.array(catalog_main['SFR'])

sSFR = np.array(catalog_main['sSFR'])
################################################################################

/var/folders/5x/2533z9ns34n_8ghxd5f6hl0c0000gn/T/ipykernel_19633/2049853064.py:4: RuntimeWarning: divide by zero encountered in log10
  Mstar_NSA = np.log10(catalog_main['ELPETRO_MASS'])


## Separate galaxies by their LSS classifications

### $V^2$

In [6]:
wall_v2 = catalog_main['vflag_V2'] == 0
void_v2 = catalog_main['vflag_V2'] == 1
edge_v2 = catalog_main['vflag_V2'] == 2
out_v2 = catalog_main['vflag_V2'] == 9

### VoidFinder

In [7]:
wall_vf = catalog_main['vflag_VF'] == 0
void_vf = catalog_main['vflag_VF'] == 1
edge_vf = catalog_main['vflag_VF'] == 2
out_vf = catalog_main['vflag_VF'] == 9

# Calculate the KS test statistic and associated $p$-value

## $M_*$

In [18]:
VF_Mstar_ks_stat, VF_Mstar_p = ks_2samp(Mstar_NSA[void_vf], Mstar_NSA[wall_vf])

V2_Mstar_ks_stat, V2_Mstar_p = ks_2samp(Mstar_NSA[void_v2], Mstar_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_Mstar_ks_stat, VF_Mstar_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_Mstar_ks_stat, V2_Mstar_p))

VF KS statistic = 0.086    p-value = 0.000e+00
V2 KS statistic = 0.006    p-value = 0.112


## $M_r$

In [19]:
VF_rabsmag_ks_stat, VF_rabsmag_p = ks_2samp(rabsmag_NSA[void_vf], rabsmag_NSA[wall_vf])

V2_rabsmag_ks_stat, V2_rabsmag_p = ks_2samp(rabsmag_NSA[void_v2], rabsmag_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_rabsmag_ks_stat, VF_rabsmag_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_rabsmag_ks_stat, V2_rabsmag_p))

VF KS statistic = 0.060    p-value = 2.140e-192
V2 KS statistic = 0.006    p-value = 0.101


## Color

### $u - r$

In [20]:
VF_ur_ks_stat, VF_ur_p = ks_2samp(u_r_NSA[void_vf], u_r_NSA[wall_vf])

V2_ur_ks_stat, V2_ur_p = ks_2samp(u_r_NSA[void_v2], u_r_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_ur_ks_stat, VF_ur_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_ur_ks_stat, V2_ur_p))

VF KS statistic = 0.131    p-value = 0.000e+00
V2 KS statistic = 0.006    p-value = 0.128


### $g - r$

In [21]:
VF_gr_ks_stat, VF_gr_p = ks_2samp(g_r_NSA[void_vf], g_r_NSA[wall_vf])

V2_gr_ks_stat, V2_gr_p = ks_2samp(g_r_NSA[void_v2], g_r_NSA[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_gr_ks_stat, VF_gr_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_gr_ks_stat, V2_gr_p))

VF KS statistic = 0.126    p-value = 0.000e+00
V2 KS statistic = 0.005    p-value = 0.342


## SFR

In [22]:
VF_SFR_ks_stat, VF_SFR_p = ks_2samp(SFR[void_vf], SFR[wall_vf])

V2_SFR_ks_stat, V2_SFR_p = ks_2samp(SFR[void_v2], SFR[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_SFR_ks_stat, VF_SFR_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_SFR_ks_stat, V2_SFR_p))

VF KS statistic = 0.096    p-value = 0.000e+00
V2 KS statistic = 0.003    p-value = 0.763


## sSFR

In [23]:
VF_sSFR_ks_stat, VF_sSFR_p = ks_2samp(sSFR[void_vf], sSFR[wall_vf])

V2_sSFR_ks_stat, V2_sSFR_p = ks_2samp(sSFR[void_v2], sSFR[wall_v2])

print('VF KS statistic = {:.3f}    p-value = {:.3e}'.format(VF_sSFR_ks_stat, VF_sSFR_p))
print('V2 KS statistic = {:.3f}    p-value = {:.3f}'.format(V2_sSFR_ks_stat, V2_sSFR_p))

VF KS statistic = 0.133    p-value = 0.000e+00
V2 KS statistic = 0.004    p-value = 0.610
